# Delete Redshift Notebook

In [10]:
import boto3
import time
import pandas as pd

In [24]:
# load configuration file

from configparser import ConfigParser

config = ConfigParser()
config.read_file(open('dwh.cfg'))

KEY=config.get('AWS','key')
SECRET= config.get('AWS','secret')
DB_HOST = config.get("CLUSTER","HOST")
DB_ROLE_NAME = config.get("CLUSTER", "DB_ROLE_NAME")


In [12]:
# Create Redshift client
from boto3 import client

redshift = client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

In [13]:
# Create IAM Client

from boto3 import client

iam = client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

In [18]:
# delete Redshift resources
try:
    redshift.delete_cluster( ClusterIdentifier=DB_HOST,  SkipFinalClusterSnapshot=True)
except Exception as e:
    print(e)

An error occurred (InvalidClusterState) when calling the DeleteCluster operation: There is an operation running on the Cluster. Please try to delete it at a later time.


In [21]:
# verify deletion

def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])



starttime=time.time()
while (redshift.describe_clusters(ClusterIdentifier=DB_HOST)['Clusters'][0] == 'deleting') == True:
    print("Checking every minute for redshift deletion...")
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))

try:    
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DB_HOST)['Clusters'][0]
    prettyRedshiftProps(myClusterProps)
except ClusterNotFoundFault as e:
    print(e)


ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster dwhcluster not found.

In [25]:
#remove roles and policies
iam.detach_role_policy(RoleName=DB_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DB_ROLE_NAME)

{'ResponseMetadata': {'RequestId': 'b9c431e4-cb39-43ed-bdd5-9a46ab874e51',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b9c431e4-cb39-43ed-bdd5-9a46ab874e51',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Thu, 28 May 2020 04:12:36 GMT'},
  'RetryAttempts': 0}}